In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, AncillaRegister, IBMQ
from qiskit.circuit.library import QFT, GraphState, GroverOperator
from qiskit.qasm import pi
from utils import *


from benchmarks import grover, qaoa, ghz, qft, qft_entangled, shor, w_state, qpe_inexact, qpe_exact, graph_state, vqe, deutsch_jozsa

In [8]:
provider = IBMQ.load_account()
backend = provider.backend.ibmq_santiago
c_map = backend.configuration().coupling_map

In [ ]:
backend.configuration().basis_gates

In [10]:
benchmarks = [grover, qaoa, ghz, qft, qft_entangled, w_state, qpe_inexact, qpe_exact, vqe, deutsch_jozsa]

In [11]:
num_qubits_test = 5
n_min = 3
n_max = 7
opt_level = 2

In [12]:
max_n_arch = max(max(c_map))
for benchmark in benchmarks:
    for n in range(n_min, n_max):
        qc = benchmark.create_circuit(n)
        compiled_without_architecure = get_compiled_circuit(qc, opt_level=opt_level)
        save_as_qasm(compiled_without_architecure, n)
        if (n < max_n_arch):
            compiled_with_architecture = get_compiled_circuit(qc, opt_level=opt_level, c_map=c_map)
            save_as_qasm(compiled_with_architecture, n, True)

optimal function value: 2.0
optimal value: [1. 0. 0.]
status: SUCCESS
optimal function value: 4.0
optimal value: [0. 1. 1. 0.]
status: SUCCESS
optimal function value: 4.0
optimal value: [0. 1. 1. 0. 0.]
status: SUCCESS
optimal function value: 6.0
optimal value: [0. 1. 1. 1. 0. 0.]
status: SUCCESS
optimal function value: 2.0
optimal value: [1. 0. 0.]
status: SUCCESS
optimal function value: 4.0
optimal value: [0. 1. 1. 0.]
status: SUCCESS


C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\scipy\optimize\optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


optimal function value: 4.0
optimal value: [0. 1. 1. 0. 0.]
status: SUCCESS
optimal function value: 4.0
optimal value: [1. 1. 0. 0. 0. 0.]
status: SUCCESS


Grover Test

In [ ]:
a = grover.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

QAOA Test

In [ ]:
a = qaoa.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

GHZ Test

In [ ]:
a = ghz.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

QFT Test

In [ ]:
a = qft.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

QFT Entangled Test

In [ ]:
a = qft_entangled.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

W-State Test

In [ ]:
a = w_state.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

QPE Exact and QPE Inexact Test

In [ ]:
a = qpe_exact.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

a = qpe_inexact.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

Graph State Test

In [ ]:
a = graph_state.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)


Shor Test

In [ ]:
a = shor.create_circuit(15)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, None)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, None, True)

QAE Test

In [ ]:
import ae

In [ ]:
a = ae.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)


DJ Test

In [4]:
deutsch_jozsa.create_circuit(15).draw()

┌───┐     ┌──────────┐┌───┐┌─┐                                          
 q_0: ┤ H ├─────┤0         ├┤ H ├┤M├──────────────────────────────────────────
      ├───┤     │          │├───┤└╥┘┌─┐                                       
 q_1: ┤ H ├─────┤1         ├┤ H ├─╫─┤M├───────────────────────────────────────
      ├───┤     │          │├───┤ ║ └╥┘┌─┐                                    
 q_2: ┤ H ├─────┤2         ├┤ H ├─╫──╫─┤M├────────────────────────────────────
      ├───┤     │          │├───┤ ║  ║ └╥┘┌─┐                                 
 q_3: ┤ H ├─────┤3         ├┤ H ├─╫──╫──╫─┤M├─────────────────────────────────
      ├───┤     │          │├───┤ ║  ║  ║ └╥┘┌─┐                              
 q_4: ┤ H ├─────┤4         ├┤ H ├─╫──╫──╫──╫─┤M├──────────────────────────────
      ├───┤     │          │├───┤ ║  ║  ║  ║ └╥┘┌─┐                           
 q_5: ┤ H ├─────┤5         ├┤ H ├─╫──╫──╫──╫──╫─┤M├───────────────────────────
      ├───┤     │          │├───┤ ║  ║  ║  ║  ║ └╥┘┌─┐                        
 q_6: ┤ H ├─────┤6         ├┤ H ├─╫──╫──╫──╫──╫──╫─┤M├────────────────────────
      ├───┤     │          │├───┤ ║  ║  ║  ║  ║  ║ └╥┘┌─┐                     
 q_7: ┤ H ├─────┤7         ├┤ H ├─╫──╫──╫──╫──╫──╫──╫─┤M├─────────────────────
      ├───┤     │   Oracle │├───┤ ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐                  
 q_8: ┤ H ├─────┤8         ├┤ H ├─╫──╫──╫──╫──╫──╫──╫──╫─┤M├──────────────────
      ├───┤     │          │├───┤ ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐               
 q_9: ┤ H ├─────┤9         ├┤ H ├─╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├───────────────
      ├───┤     │          │├───┤ ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐            
q_10: ┤ H ├─────┤10        ├┤ H ├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├────────────
      ├───┤     │          │├───┤ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐         
q_11: ┤ H ├─────┤11        ├┤ H ├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├─────────
      ├───┤     │          │├───┤ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐      
q_12: ┤ H ├─────┤12        ├┤ H ├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├──────
      ├───┤     │          │├───┤ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐   
q_13: ┤ H ├─────┤13        ├┤ H ├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├───
      ├───┤     │          │├───┤ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐
q_14: ┤ H ├─────┤14        ├┤ H ├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├
      ├───┤┌───┐│          │└───┘ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘
q_15: ┤ X ├┤ H ├┤15        ├──────╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─
      └───┘└───┘└──────────┘      ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ 
c: 15/════════════════════════════╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩═
                                  0  1  2  3  4  5  6  7  8  9  10 11 12 13 14

In [9]:
a = deutsch_jozsa.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

TranspilerError: 'Number of qubits (6) in dj is greater than maximum (5) in the coupling_map'